<a href="https://colab.research.google.com/drive/1KIH6qjc6nEfLRXfLkIEwBjjGPlvs4_AQ?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para executar isso, pressione "*Runtime*" e pressione "*Executar tudo*" em uma instância **gratuita** do Tesla T4 Google Colab!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Entre no Discord se precisar de ajuda + ⭐ <i>Marque-nos com uma estrela <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>




Aqui a documentação https://docs.unsloth.ai/get-started/beginner-start-here

### Instalação

In [ ]:
%%capture


# downgrade trl to 0.15.2 due STTFTTrainer issues: https://github.com/unslothai/notebooks/pull/29
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

# fazer o downgrade de versão tambem funciona:
# !pip install "unsloth==2025.3.18" "unsloth_zoo==2025.3.16"

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Orenguteng/Llama-3.1-8B-Lexi-Uncensored-V2",
    max_seq_length = max_seq_length,
    dtype = dtype
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Agora adicionamos adaptadores LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Preparar Dados
Para carregar o dataset escolha uma das opcões (mude para True)



In [ ]:
def format_chat(conversation):
  system_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|>"
  user_prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{conversation['input']}<|eot_id|>"
  assistant_response = f"<|start_header_id|>assistant<|end_header_id|>\n\n{conversation['output']}<|eot_id|>"

  return {"text": system_prompt + user_prompt + assistant_response}

from datasets import load_dataset

# Para carregar o dataset escolha uma das três opções (mude para True)

# Carregar do github gist:
if True:
  dataset = load_dataset("json", data_files={"train": "https://gist.githubusercontent.com/HonraVT/45319e12ecb462212ebad40fd41d99fc/raw/30b3d378351a232f9309f6635b8b13e0c38a9da0/jose_dataset_2405.json"}, split = "train")
  dataset = dataset.map(format_chat)

# Carregar direto no colab:
if False:
  # Rode essa celula uma vez, depois no menu da esquerda clique na no icone de pasta, depois no icone de pasta com dois pontos
  # depois na pasta <dataset> clique com o botão direito e em upload e envie o arquivo que deve ter o nome jose_dataset_2405.json
  # Rode novamente
  from pathlib import Path
  path = Path("/content/dataset/jose_dataset_2405.json")
  if path.exists():
    dataset = load_dataset("json", data_files={"train": "dataset/jose_dataset_2405.json"}, split = "train")
    print("Dataset carregado!")
    dataset = dataset.map(format_chat)
  else:
    !mkdir dataset
    print("Faça o upload manualmente e rode novamente.")

# Carregar do huggingface
if False:
  %cd /content
  !huggingface-cli download fuze-eletrik/jose_posts jose_dataset.json --repo-type dataset --local-dir ./dataset
  dataset = load_dataset("json", data_files={"train": "dataset/jose_dataset_2405.json"}, split = "train")
  dataset = dataset.map(format_chat)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2405 [00:00<?, ? examples/s]

veja como o template de conversação transformou a conversa.

**[Aviso]** O modelo de bate-papo padrão do Llama 3.1 Instruct adiciona `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, então tambem adcionei isso aos dados de treinamento conforme o tutorial.

In [ ]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSeus cachorros entendam uma coisa<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nRestanho joga ETS2 e vocês nao pentelhsk ele Não importa o ano do jogo mas sim ele divertir,  relaxar. Tanto q tem gente q joga mega drive hoje Vcee jilgam minga família acusando e caluniando de coisas q vces supõe q vao  fazer  comigo no futuro. Calunia É CRIME<|eot_id|>'

<a name="Train"></a>
### Treine o modelo
Agora vamos usar o `SFTTrainer` do Huggingface TRL! Mais documentos aqui: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).  Vamos configurar 2 epochs.

Para saber mais sobre [epochs](https://docs.unsloth.ai/get-started/fine-tuning-guide#epochs)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        # save_strategy = "epoch",
        save_strategy = "steps",
        save_steps = 300,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2405 [00:00<?, ? examples/s]

Verificamos que o mascaramento é realmente feito:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSeus cachorros entendam uma coisa<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nRestanho joga ETS2 e vocês nao pentelhsk ele Não importa o ano do jogo mas sim ele divertir,  relaxar. Tanto q tem gente q joga mega drive hoje Vcee jilgam minga família acusando e caluniando de coisas q vces supõe q vao  fazer  comigo no futuro. Calunia É CRIME<|eot_id|>'

In [ ]:
# @title Mostrar estatísticas de memória atuais
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.555 GB of memory reserved.


### Inicie o Treinamento!
A meta é um training Loss de a 1.0 a 0,5. Muito abaixo disso o modelo não funcionará corretamente (traine loss zerado ou negativo definitivamente há problemas) [Avaliando](https://docs.unsloth.ai/get-started/fine-tuning-guide#id-6.-training--evaluation)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,405 | Num Epochs = 1 | Total steps = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.668500
2,3.891000
3,3.540700
4,3.628200
5,3.306800


In [ ]:
# @title Mostrar status final de tempo e memoria
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2277.3144 seconds used for training.
37.96 minutes used for training.
Peak reserved memory = 7.57 GB.
Peak reserved memory for training = 2.015 GB.
Peak reserved memory % of max memory = 51.353 %.
Peak reserved memory for training % of max memory = 13.669 %.


<a name="Inferência"></a>
### Inferência
Vamos executar o modelo! Você pode alterar a instrução e a entrada - deixe a saída em branco!

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "quanto é 4 + 4?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 1.5, min_p = 0.1
    )
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nquanto é 4 + 4?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n4 + 4 = 8<|eot_id|>']

### Conversão GGUF / llama.cpp
Para salvar em `GGUF` / `llama.cpp`, clonamos `llama.cpp` e salvamos como padrão em `q8_0`. Permitimos todos os métodos como `q4_k_m`. Use `save_pretrained_gguf` para salvar localmente e `push_to_hub_gguf` para fazer upload para HF.

Alguns métodos quantitativos suportados (lista completa em nossa [página Wiki](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Conversão rápida. Alto uso de recursos, mas geralmente aceitável.
* `q4_k_m` - Recomendado. Usa Q6_K para metade dos tensores attention.wv e feed_forward.w2, senão Q4_K.
* `q5_k_m` - Recomendado. Usa Q6_K para metade dos tensores attention.wv e feed_forward.w2, caso contrário Q5_K.

###Para salvar manualmente

Após completar a célula abaixo vá no icone da pasta no menu esquerdo e clique na pasta `model` e clique com o botão direito no arquivo `unsloth.Q8_0.gguf` (que é seu modelo finetunado e pronto para usar) e depois em **fazer download** (é um pouco bugado demora para aparecer a janela do windows para escolher onde salvar devido ao tamanho do arquivo)

###Ideal é salvar no huggingface
Crie uma conta no huggingface, gere seu token [AQUI](https://huggingface.co/settings/tokens) e salve seu modelo nas duas principais quantizações `Q8_0` boa precisão e `q4_k_m` metade da precisão mas de rápida inferência.


In [ ]:

# Salva com 8bit Q8_0 quantização localmente:
# Apos completar essa celula
if True: model.save_pretrained_gguf("model", tokenizer,)

# E troque hf pelo seu nome de usuário e model pelo nome que deseja dar ao modelo
# LEMBRE-SE de acessar https://huggingface.co/settings/tokens para obter um token!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF full precision
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Salvar em várias opções GGUF - muito mais rápido!
# OTIMIZADO! o ideal é salver no huggingface da forma abaixo:
if False:
    model.push_to_hub_gguf(
        "fuze-eletrik/llama-3.1-8B-lexi-uncensored-V2-jose-GGUF", # Troque fuze-eletrik pelo seu nome de usuário!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0",],
        token = "hf_wasdwasdwasdwasdwasdwasd", # Troque pelo seu token!
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.59 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:02<00:01, 10.61it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:17<00:00,  4.30s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

###Opcional, Carregar, testar e salvar checkpoints

Se voçe rodou 2 epochs com o dataset de exemplo observe que na pasta `outputs` há duas pastas, uma chamada `checkpoint-300` e outra `checkpoint-600` então voçe pode testar e salvar qualquer um dessas estagios de treinamento.

Porem devido a limitação de espaço do google colab voçe tera que apagar a pasta `model`:

Mude para True

In [ ]:
if False:
  import shutil
  shutil.rmtree("/content/model")

In [ ]:
# carregar o checkpoint escolhido
if False:
  model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="outputs/checkpoint-xxx", # Substitua xxx pelo número do passo de treinamento desejado
        max_seq_length=max_seq_length,
        dtype=dtype
    )

# inferencia
if False:
  tokenizer = get_chat_template(
      tokenizer,
      chat_template="llama-3.1",
      )
  FastLanguageModel.for_inference(model)
  messages = [
      {"role": "user", "content": "cristione gosta de vc?"},
      ]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt",
      ).to("cuda")
  outputs = model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1)
  print(tokenizer.batch_decode(outputs))

# salvar checkpoint
if False:
    model.push_to_hub_gguf(
        "fuze-eletrik/llama-3.1-8B-lexi-uncensored-V2-jose-GGUF", # Troque fuze-eletrik pelo seu nome de usuário!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0",],
        token = "hf_wasdwasdwasdwasdwasdwasd", # Troque pelo seu token!
    )




###Opcional, Salvar para o Google Drive
priveiro execute o codigo abaixo, o colab vai alertar que isso pode ser perigoso, ignore e fa;a a autenticação

Mude para True em ambos:




In [ ]:
if False:
  from google.colab import drive
  drive.mount('/content/drive')

Agora rode abaixo para copiar o arquivo

In [ ]:
if False:
  import shutil
  # Nome do arquivo existente no Colab
  arquivo_existente = "/content/model/unsloth.Q8_0.gguf"

  # Caminho de destino no Google Drive
  destino = '/content/drive/My Drive/unsloth.Q8_0.gguf'

  # Copiar o arquivo para o Drive
  shutil.copy(arquivo_existente, destino)

<a name="Save"></a>
### Opcional, salvar o adaptador LORA
Para salvar o modelo final como adaptadores LoRA, use o comando `push_to_hub` para salvar online ou o comando `save_pretrained` para salvar localmente.

**[NOTA]** Isso salva SOMENTE os adaptadores LoRA, e não o modelo completo.

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Agora, use o arquivo `model-unsloth.gguf` ou o arquivo `model-unsloth-Q4_K_M.gguf` em llama.cpp ou um sistema baseado em UI como Jan ou Open WebUI. Você pode instalar Jan [aqui](https://github.com/janhq/jan) e Open WebUI [aqui](https://github.com/open-webui/open-webui)

E pronto! Se você tiver alguma dúvida sobre Unsloth, temos um canal [Discord](https://discord.gg/unsloth)! Se você encontrar algum bug ou quiser se manter atualizado com as últimas novidades do LLM, ou precisar de ajuda, participar de projetos etc., sinta-se à vontade para participar do nosso Discord!

Alguns outros links:
1. Caderno de conversação Llama 3.2. [Colab grátis](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Salvando ajustes finos no Ollama. [Notebook grátis](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Ajuste fino do Llama 3.2 Vision - Caso de uso de radiografia. [Colab grátis](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. Veja os notebooks para DPO, ORPO, pré-treinamento contínuo, ajuste fino de conversação e muito mais em nossa [documentação](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Entre no Discord se precisar de ajuda + ⭐️ <i>Marque-nos com uma estrela <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
